## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os, sys

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [15]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bredasdorp,ZA,-34.53,20.04,64.40,88,9,8.05,clear sky
1,1,Marsh Harbour,BS,26.54,-77.06,81.28,79,10,11.39,clear sky
2,2,Sagua La Grande,CU,22.81,-80.08,87.80,66,75,13.87,broken clouds
3,3,Nanortalik,GL,60.14,-45.24,29.64,87,0,26.78,clear sky
4,4,Ushuaia,AR,-54.80,-68.30,57.20,58,75,11.41,light rain


In [16]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Marsh Harbour,BS,26.54,-77.06,81.28,79,10,11.39,clear sky
2,2,Sagua La Grande,CU,22.81,-80.08,87.80,66,75,13.87,broken clouds
8,8,Key Largo,US,25.09,-80.45,84.00,1,52,10.00,light rain
11,11,Kankan,GN,10.39,-9.31,80.11,44,29,3.78,scattered clouds
13,13,Hambantota,LK,6.12,81.12,78.80,100,90,1.12,overcast clouds
16,16,Gurupi,BR,-11.73,-49.07,88.84,51,27,5.95,moderate rain
18,18,Marabba,SD,12.35,32.18,88.61,36,70,6.15,broken clouds
21,21,Kalmunai,LK,7.42,81.82,81.03,74,54,5.64,broken clouds
24,24,Neuquen,AR,-38.95,-68.06,89.60,18,0,5.82,clear sky
26,26,Wattegama,LK,6.80,81.48,80.60,94,90,3.36,overcast clouds


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                226
City                   226
Country                226
Lat                    226
Lng                    226
Max Temp               226
Humidity               226
Cloudiness             226
Wind Speed             226
Current Description    226
dtype: int64

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                226
City                   226
Country                226
Lat                    226
Lng                    226
Max Temp               226
Humidity               226
Cloudiness             226
Wind Speed             226
Current Description    226
dtype: int64

In [23]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Marsh Harbour,BS,81.28,clear sky,26.54,-77.06,
2,Sagua La Grande,CU,87.80,broken clouds,22.81,-80.08,
8,Key Largo,US,84.00,light rain,25.09,-80.45,
11,Kankan,GN,80.11,scattered clouds,10.39,-9.31,
13,Hambantota,LK,78.80,overcast clouds,6.12,81.12,
16,Gurupi,BR,88.84,moderate rain,-11.73,-49.07,
18,Marabba,SD,88.61,broken clouds,12.35,32.18,
21,Kalmunai,LK,81.03,broken clouds,7.42,81.82,
24,Neuquen,AR,89.60,clear sky,-38.95,-68.06,
26,Wattegama,LK,80.60,overcast clouds,6.80,81.48,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [25]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head()
#hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Marsh Harbour,BS,81.28,clear sky,26.54,-77.06,Abaco Beach Resort
2,Sagua La Grande,CU,87.80,broken clouds,22.81,-80.08,Hotel Amaro
8,Key Largo,US,84.00,light rain,25.09,-80.45,Bayside Inn Key Largo
11,Kankan,GN,80.11,scattered clouds,10.39,-9.31,Hotel Bate
13,Hambantota,LK,78.80,overcast clouds,6.12,81.12,Bungalow 63


In [26]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))